# Final Project: NeuroDump - thoughts organizer

## 1. Environment setup

### 1.1. Environment

In [1]:
#!pip install -r requirements.txt --- UPDATE THIS AT THE END

### 1.2. Tools and Libraries

### 1.3. Custom functions

# ### IMPORT

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import nltk
from transformers import AutoTokenizer
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from pathlib import Path
from IPython.display import Markdown, display
import hdbscan
import numpy as np
import json
from collections import defaultdict
import subprocess

nltk.download('punkt')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

[nltk_data] Downloading package punkt to /Users/work/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Craft pipeline

   1. Input unstructured document file
      1. Config paths
      2. Check if the input is already processed or is a new file
      3. Only continue if it's a new file
   2. Split document into chunks of tokens
   3. Embed all chunks
      1. sentence-transformers (all-MiniLM-L6-v2)
      2. save embeddings into Chroba DB
      3. keep metada to track original document file for each chunk
   4. Cluster the embeddings by semantic similarity (based on all existing vectors in Chroma DB, not only from new file)
   5. Label each cluster based on existing labels in Chroma DB, if no match, create a new label
      1. maybe prompt to LLM to generate labels?
   6. Rewrite each labeled-cluster to connect all the chunks and create a coherent sub-document 
      1. use LLM for that.
      2. output should be like: sub-document 1 (label 1), sub-document 2 (label 2)
   7. Store each sub-document in correspondent label as a new file
      1. For test, generate a new folder locally and save the sub-document as .txt file in the folder with the correspondent label
      2. Use Notion API, the sub-document should be a new file in a Notion database, with the correpondent Label and metadata
   8. Add more documents in bulk and generate insights for simple EDA
       1.  Number of labels, sub-documents in each label, etc.

### 1. Input unstructured document
Incremental version. It will process only new files.

In [3]:
# Config paths
FOLDER_PATH = "./data/mock_notes/"          #files
LOG_PATH = "./data/processed_files.txt"     #list of processed files
PERSIST_DIR = "./data/chroma_db"            #embeddings database (store vectorized chunks)
CLUSTER_DIR = "./data/cluster_dir/"         #clusters database

# Create directories if they don't exist
os.makedirs(FOLDER_PATH, exist_ok=True)
os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)
os.makedirs(PERSIST_DIR, exist_ok=True)
os.makedirs(CLUSTER_DIR, exist_ok=True)

In [4]:
# Initialize embedding model
HF_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

#### 1.2. Check if the input is already processed or is a new file

In [5]:
def get_processed_files():
    """Load list of already processed files from log."""
    if os.path.exists(LOG_PATH):
        with open(LOG_PATH, "r") as f:
            return set(line.strip() for line in f)
    return set()


def get_new_files():
    """Get list of new .txt files that haven't been processed yet."""
    if not os.path.exists(FOLDER_PATH):
        return []
    
    all_files = [f for f in os.listdir(FOLDER_PATH) if f.endswith(".txt")]
    processed_files = get_processed_files()
    return [f for f in all_files if f not in processed_files]

### 2. Split document into chunks of tokens

In [6]:
def count_tokens(text):
    """Count tokens in text using the embedding model's tokenizer."""
    return len(tokenizer.tokenize(text))

def is_code_like(sentence):
    """Detect if a sentence contains code-like patterns."""
    code_keywords = ["from ", "import ", "def ", "return ", "class ", "for ", "if ", "else", "while ", "try:", "except", "print("]
    code_symbols = ["=", "()", "[]", "{}", "->", "::"]
    sentence_lower = sentence.strip().lower()
    return (
        any(sentence_lower.startswith(k) for k in code_keywords) or
        any(sym in sentence for sym in code_symbols)
    )


# ### CHUNKING LOGIC
def chunk_text(text, source_name, min_tokens=40, max_tokens=100, inspect=False):
    """Split text into semantic chunks with token limits."""
    raw_blocks = re.split(r"\n\s*\n", text.strip())                    # Split by double newlines
    chunks, current_chunk, current_tokens = [], [], 0                  # Initialize chunking variables
    metadata = []                                                      # Store metadata for each chunk

    for block in raw_blocks:                                           # Process each text block
        sentences = nltk.sent_tokenize(block.strip())                  # Split block into sentences

        for sentence in sentences:                                     # Process each sentence
            sentence = sentence.strip()
            tokens = count_tokens(sentence)                            # Count tokens in sentence

            # Handle code-like sentences separately
            if is_code_like(sentence):                                 # Check if sentence contains code
                if current_chunk:                                      # Save current chunk if exists
                    chunks.append(" ".join(current_chunk))
                    metadata.append({"source": source_name})
                    current_chunk, current_tokens = [], 0
                chunks.append(sentence)                                # Add code as separate chunk
                metadata.append({"source": source_name})
                continue

            # Handle conversation transitions
            if any(sentence.lower().startswith(w) for w in ["btw", "wait", "oh", "then", "also", "now", "next"]):
                if current_chunk:                                      # Break chunk at transition words
                    chunks.append(" ".join(current_chunk))
                    metadata.append({"source": source_name})
                    current_chunk, current_tokens = [], 0

            # Check if adding sentence would exceed max tokens
            if current_tokens + tokens > max_tokens and current_tokens >= min_tokens:
                chunks.append(" ".join(current_chunk))                # Save current chunk
                metadata.append({"source": source_name})
                current_chunk, current_tokens = [], 0                 # Reset for new chunk

            current_chunk.append(sentence)                            # Add sentence to current chunk
            current_tokens += tokens                                  # Update token count

        # Save any remaining chunk after block processing
        if current_chunk:
            chunks.append(" ".join(current_chunk))
            metadata.append({"source": source_name})
            current_chunk, current_tokens = [], 0

    # if inspect:                                                       # Optional: print chunks for debugging
    #     for i, chunk in enumerate(chunks):
    #         print(f"\n=== Chunks ===")
    #         print(f"\nChunk {i+1} ({count_tokens(chunk)} tokens):\n{chunk}")

    return chunks, metadata

### 3. Embed all chunks

#### 3.1. Save embeddings into Chroma DB

In [7]:
# ### VECTOR STORE MANAGEMENT
class VectorStoreManager:
    def __init__(self):
        self.persist_dir = PERSIST_DIR                              # Store database directory path
        self.embeddings = HF_embeddings                          # Store embedding model
        self.vectorstore = self._load_existing()                    # Load existing DB or set to None

    def _load_existing(self):
        """Load existing Chroma database if it exists."""
        try:
            if os.path.exists(self.persist_dir) and os.listdir(self.persist_dir):
                print(f"\n=== Loading existing Chroma DB from {self.persist_dir}")
                return Chroma(
                    persist_directory=self.persist_dir,
                    embedding_function=self.embeddings,
                    collection_name="default"
                )
        except Exception as e:
            print(f"\n=== Error loading existing Chroma DB: {e}")
        
        print("\n=== No existing Chroma DB found. Will create new one.")
        return None

    def add_chunks(self, chunks, metadata):
        """Add text chunks to vector store with embeddings."""
        if not self.vectorstore:                                     # Create new database if none exists
            print(f"\n=== Creating new Chroma DB at {self.persist_dir}")
            self.vectorstore = Chroma.from_texts(
                texts=chunks,
                metadatas=metadata,
                embedding=self.embeddings,
                persist_directory=self.persist_dir,
                collection_name="default"
            )
        else:                                                        # Add to existing database
            self.vectorstore.add_texts(texts=chunks, metadatas=metadata)


    def get_chunks_with_embeddings(self, fname):
        """Retrieve chunks, embeddings, and metadata for a specific file."""
        results = self.vectorstore.get(
            where={"source": fname},
            include=["documents", "embeddings", "metadatas"]
        )
        chunks = results.get('documents', [])               # Extract text chunks
        embeddings_list = results.get('embeddings', [])     # Exttact embeddings
        metadatas = results.get('metadatas', [])            # Extract metadata
    
        return chunks, embeddings_list, metadatas        

### 4. Cluster the embeddings by semantic similarity

In [8]:
# ### CLUSTERING LOGIC
class ChunkClusterizer:
    def __init__(self, chunks, chunk_embeddings, metadata, fname):
        self.chunks = chunks                                         # Store text chunks
        self.embeddings = np.array(chunk_embeddings)                 # Convert embeddings to numpy array for ML operations
        self.metadata = metadata                                     # Save metadata       
        self.fname = fname
        self.labels = []                                             # Initialize cluster labels
        self.clusters = defaultdict(list)                            # cluster_id -> list of dicts {chunk, metadata}


    def cluster_chunks(self, min_cluster_size=2, min_samples=1):
        """Cluster chunks using HDBSCAN algorithm."""
        
        # Handle edge cases
        if len(self.chunks) < min_cluster_size:                      # Not enough chunks to cluster
            print(f"\n=== Not enough chunks to cluster ({len(self.chunks)} < {min_cluster_size}) ===")
            self.labels = [-1] * len(self.chunks)                   # Assign all to noise
            return self.labels

        # Ensure embeddings are 2D
        if self.embeddings.ndim == 1:                               # Reshape 1D embeddings to 2D
            self.embeddings = self.embeddings.reshape(1, -1)
            
        #print(f"\n===> Clustering {len(self.chunks)} chunks with embeddings shape: {self.embeddings.shape}")
            

        # Perform clustering
        clusterizer = hdbscan.HDBSCAN(                                     # Initialize HDBSCAN clustering algorithm
            min_cluster_size=min_cluster_size,                             # Minimum number of points in a cluster
            min_samples=min_samples,                                       # Minimum samples in neighborhood for core point
            metric='euclidean'                                             # Distance metric for clustering
        )
        self.labels = clusterizer.fit_predict(self.embeddings)             # Fit model and predict cluster labels = generate cluster labels

        # Display results
        num_clusters = len(set(self.labels)) - (1 if -1 in self.labels else 0)  # Count clusters (exclude noise)
        print(f"\n=== Found {num_clusters} clusters")
        print("\n=== Cluster distribution ===")
        unique, counts = np.unique(self.labels, return_counts=True)
        for label, count in zip(unique, counts):
            cname = "Noise" if label == -1 else f"Cluster {label}"
            print(f"{cname}: {count} chunks")    
                    
        # Show cluster contents
        for cluster_id in sorted(set(self.labels)):
            cluster_name = "Noise" if cluster_id == -1 else f"Cluster {cluster_id}"
            print(f"\n=== {cluster_name} ===")

            for i, label in enumerate(self.labels):
                if label == cluster_id:
                    # Save the chunk into the cluster dictionary with global ID
                    self.clusters[str(cluster_id)].append({
                        "chunk_id": i,                      # Global ID
                        "chunk": self.chunks[i],
                        "metadata": self.metadata[i]
                    })

                    # Display clearly: Chunk [global id]
                    display(Markdown(f"**Chunk {i}:** {self.chunks[i]}"))
            
        return self.labels                                   # Return cluster labels
        
    def save_clusters(self):
        """Save clusters with metadata"""
        out_path = os.path.join(CLUSTER_DIR, self.fname.replace(".txt", "_clusters.json"))
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(self.clusters, f, indent=2, ensure_ascii=False)
        print(f"\n=== Saved clusters to: {out_path}")

### PROCESSING PIPELINE - steps 1 to 4 for now...

In [9]:
# ### MAIN PROCESSING PIPELINE
def process_file(fname):
    """Process a single file through the complete pipeline."""
    print(f"\n\n=============== PROCESSING FILE: {fname} ===============")

    # Step 1: Read file
    with open(os.path.join(FOLDER_PATH, fname), "r", encoding="utf-8") as f:
        text = f.read()

    # Step 2: Create chunks
    chunks, metadata = chunk_text(text, fname, inspect=True)        # Split text into chunks
    print(f"\n=== Created {len(chunks)} chunks")

    # Step 3: Store in vector database
    vector_manager = VectorStoreManager()    # Initialize vector store
    vector_manager.add_chunks(chunks, metadata)                     # Add chunks with embeddings
    print(f"\n=== Stored {len(chunks)} chunks in Chroma DB")
    
    # Step 4: Mark as processed
    with open(LOG_PATH, "a") as log:                                # Log processed file
        log.write(fname + "\n")

    # Step 5: Cluster chunks
    print(f"\n=== CLUSTERING CHUNKS FROM {fname}")
    chunks, embeddings, metadatas = vector_manager.get_chunks_with_embeddings(fname) # Retrieve stored data

    clusterizer = ChunkClusterizer(chunks, embeddings, metadatas, fname)             # Initialize clustering
    clusterizer.cluster_chunks()                                            # Perform clustering
    clusterizer.save_clusters()                                             # Save clusters


def process_all_new():
    """Process all new files in the input directory."""
    new_files = get_new_files()                                     # Get list of unprocessed files
    
    if not new_files:                                               # Exit early if no new files
        print("\n=== No new files to process.")
        return
    
    print(f"\n=== Found {len(new_files)} new files: {new_files}")
    
    for fname in new_files:                                         # Process each new file
        process_file(fname)
    
    print(f"\n=== Completed processing {len(new_files)} files")




### FOR LATER: "MAIN GUARD" OR "ENTRY POINT PATTERN" 
# EXECUTION 
#if __name__ == "__main__":
#    process_all_new()                                               # Run the complete pipeline

### Option 1: Process all new files

In [10]:
process_all_new()


=== No new files to process.


### Option 2: Process 1 file

In [11]:
#process_file("note-1-dt-r.txt")

## Continue pipeline...
   5. Label each cluster based on existing labels in Chroma DB, if no match, create a new label
      1. maybe prompt to LLM to generate labels?
   6. Rewrite each labeled-cluster to connect all the chunks and create a coherent sub-document 
      1. use LLM for that.
      2. output should be like: sub-document 1 (label 1), sub-document 2 (label 2)
   7. Store each sub-document in correspondent label as a new file
      1. For test, generate a new folder locally and save the sub-document as .txt file in the folder with the correspondent label
      2. Use Notion API, the sub-document should be a new file in a Notion database, with the correpondent Label and metadata
   8. Add more documents in bulk and generate insights for simple EDA
       1.  Number of labels, sub-documents in each label, etc.

In [12]:
# 5. Label clusters with classifier

In [13]:
def query_ollama(model: str, prompt: str) -> str:
    process = subprocess.Popen(
        ["ollama", "run", model],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True  # so we use strings, not bytes
    )
    stdout, stderr = process.communicate(input=prompt)
    if stderr:
        print("OLLAMA STDERR:", stderr)
    return stdout

In [14]:
SYSTEM_PROMPT = """
You are a smart assistant helping organize fragmented personal notes.

The notes were previously split into chunks and clustered semantically. Each original file has its own clusters, saved as JSON files inside: ./data/cluster_dir/. Your task is to refine those clusters and rewrite them as organized sub-notes.

1. REORGANIZE CLUSTERS PER FILE:
- You may SPLIT or MERGE clusters WITHIN EACH FILE to better reflect topic boundaries.
- You may RENAME clusters with meaningful TOPIC NAMES.
- While clusters are per file, topics should be GLOBALLY CONSISTENT across files.
  Example: cluster_0 from note-1.txt and cluster_0 from note-2.txt might both refer to the same topic "decision trees".

2. GENERATE CLUSTER NAMES:
- For each cluster ID (like cluster_0, cluster_1...), assign a clear, short TOPIC NAME.
- Save the cluster-name mapping as a DICTIONARY:
  {
    "cluster_0": "decision trees",
    "cluster_1": "regularization",
    ...
  }

3. REWRITE EACH CLUSTER:
- Combine the CHUNKS within each cluster into a COHERENT NOTE.
- The output should be CLEAR, READABLE, and preserve the original tone.
- You may REPHRASE, REORDER, or EDIT content to improve clarity.
- You MUST preserve the original content. DO NOT add new content. Focus only on text clarity.

4. SAVE RESULTS:
- Each rewritten cluster note must be saved to:
  ./data/outputs/rewritten_notes/{TOPIC_NAME}/
    └── {ORIGINAL_FILE}_cluster-{ID}.txt

  Example:
  ./data/outputs/rewritten_notes/decision_trees/note-1_cluster-0.txt

- Save the CLUSTER_NAMES dictionary to:
  ./data/outputs/cluster_names.json

- Save any MERGE, SPLIT, or REORGANIZATION CHANGES you make per file to:
  ./data/outputs/LLM_recluster/{ORIGINAL_FILE}_LLM_recluster.json

5. NOTES:
- Do NOT include topic names in file names.
- All cluster names must be FOLDER-SAFE (no special characters or spaces; use underscores if needed).
- Rewritten notes should be based ONLY on CHUNKS FROM THE CORRESPONDING FILE.
"""

In [17]:
test_prompt = "Say hello"
print(query_ollama("mistral", test_prompt))

OLLAMA STDERR: ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠋ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ 
 Hello! How can I assist you today?




In [15]:
response = query_ollama("mistral", SYSTEM_PROMPT)
print(response)

OLLAMA STDERR: ⠙ ⠙ ⠹ ⠼ ⠼ ⠦ ⠦ ⠇ ⠏ ⠋ ⠋ ⠹ ⠸ ⠸ ⠴ ⠴ ⠦ ⠇ ⠇ ⠋ ⠋ ⠹ ⠸ ⠸ ⠼ ⠦ ⠧ ⠇ ⠏ ⠏ ⠋ ⠙ ⠹ ⠸ ⠴ ⠦ ⠧ ⠇ ⠇ ⠋ ⠋ ⠙ ⠸ ⠸ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠸ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠋ ⠙ ⠸ 
 To complete this task, I'll follow the steps you've outlined:

1. REORGANIZE CLUSTERS PER FILE:
   - Load all clusters from each JSON file in ./data/cluster_dir/
   - Review and analyze clusters to identify overlapping or related topics, then group them accordingly while considering topic boundaries
   - Rename the clusters with meaningful topic names that reflect their content, making sure they are globally consistent across files.
   - Save any reorganization changes in ./data/outputs/LLM_recluster/{ORIGINAL_FILE}_LLM_recluster.json (e.g., `./data/outputs/LLM_recluster/note-1.txt_LLM_recluster.json`)

2. GENERATE CLUSTER NAMES:
   - Create a dictionary that maps each cluster ID to a clear, short topic name based on its content. (e.g., `{"cluster_0": "decision trees", "cluster_1": "regularization"}`)
   - Save the cluster-name mapping as ./data

In [ ]:
def extract_json(text, start_key):
    # crude way: find the JSON blob starting with start_key, then parse it
    pattern = re.compile(rf"{start_key}\s*(:|\=)?\s*(\{{.*?\}})", re.DOTALL)
    match = pattern.search(text)
    if match:
        try:
            return json.loads(match.group(2))
        except Exception as e:
            print("Error parsing JSON:", e)
    return None

cluster_names = extract_json(llm_response, "cluster_names")
llm_recluster_changes = extract_json(llm_response, "reorganization_changes")
rewritten_notes = extract_json(llm_response, "rewritten_notes")

In [ ]:
# 6. Rewrite selected outputs with [llama 3.2]

In [ ]:
#    7. Store each sub-document in correspondent label as a new file
#      1. For test, generate a new folder locally and save the sub-document as .txt file in the folder with the correspondent label


In [ ]:
# 8.    8. Add more documents in bulk and generate insights for simple EDA
